In [3]:
import os
import sys
import csv
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix as cmat
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as report  
from sklearn.model_selection import *
from sklearn.feature_selection import *
import matplotlib.pyplot as plt
import time
import random
import numpy
from datetime import datetime
import pandas as pd
import requests
import re
import xarray as xr
import pickle

In [11]:
directory = '../data/ship'

header_length = []

for filename in os.listdir(directory):
    if filename.endswith(".ctd.txt"):
        lines = open(directory+'/'+filename).readlines()
        header_length.append(lines.index('*END OF HEADER\n')+1)
header_length

[115,
 105,
 104,
 106,
 110,
 110,
 92,
 93,
 96,
 93,
 93,
 95,
 103,
 104,
 100,
 93,
 103,
 93,
 98,
 98,
 99,
 108,
 106,
 112,
 109,
 108,
 103]

In [17]:
directory = '../data/ship'
fieldnames = ['Pressure','Temperature','Salinity','Sigma-t','Oxygen_v','Oxygen_m','Fluorescence']

i = 0
ship_data = pd.DataFrame(columns=fieldnames)

for filename in os.listdir(directory):
    each_data = pd.read_csv(directory+'/'+filename,names=fieldnames,header=header_length[i],delim_whitespace=True )
    ship_data = pd.concat([ship_data,each_data])
    i = i + 1

IndexError: list index out of range

In [18]:
ship_data

,Pressure,Temperature,Salinity,Sigma-t,Oxygen_v,Oxygen_m,Fluorescence
20.0,5.9564,32.5635,25.6588,58.3,6.95,302.5,0.752
21.0,5.9558,32.5634,25.6588,58.1,6.96,302.8,0.745
22.0,5.9559,32.5634,25.6588,58.1,6.95,302.7,0.775
23.0,5.9552,32.5635,25.6590,58.1,6.95,302.3,0.787
24.0,5.9504,32.5632,25.6593,58.1,6.94,302.2,0.762
25.0,5.9489,32.5629,25.6593,58.1,6.95,302.4,0.759
26.0,5.9502,32.5627,25.6589,58.1,6.95,302.6,0.753
27.0,5.9523,32.5629,25.6588,58.1,6.95,302.3,0.760
28.0,5.9534,32.5629,25.6587,58.1,6.94,302.3,0.755
29.0,5.9528,32.5630,25.6589,58.1,6.95,302.3,0.747


In [20]:
ship_data.to_csv(directory+'/'+'ship_data_consolidated.csv',index=False)